CLEANING COLUMNS

In [ ]:
import pandas as pd

dtype_dict = {
    'hvfhs_license_num': 'object',
    'dispatching_base_num': 'object',
    'originating_base_num': 'object',
    'request_datetime': 'object',
    'on_scene_datetime': 'object',
    'pickup_datetime': 'object',
    'dropoff_datetime': 'object',
    'PULocationID': 'float64',
    'DOLocationID': 'float64',
    'trip_miles': 'float64',
    'trip_time': 'float64',
    'base_passenger_fare': 'float64',
    'tolls': 'float64',
    'bcf': 'float64',
    'sales_tax': 'float64',
    'congestion_surcharge': 'float64',
    'airport_fee': 'float64',
    'tips': 'float64',
    'driver_pay': 'float64',
    'shared_request_flag': 'object',
    'shared_match_flag': 'object',
    'access_a_ride_flag': 'object',
    'wav_request_flag': 'object',
    'wav_match_flag': 'object'
}

sampled_chunks = []

file_path = "./2021 Dataset/2021.csv"

sampling_fraction = 0.25

for chunk in pd.read_csv(file_path, dtype=dtype_dict, chunksize=1000000):
    sampled_chunk = chunk.sample(frac=sampling_fraction)
    sampled_chunks.append(sampled_chunk)

reduced_dataset = pd.concat(sampled_chunks, ignore_index=True)

reduced_dataset.to_csv("./2021 Dataset/reduced_2021.csv", index=False)

print("Reduced dataset created and saved successfully.")

In [ ]:
import pandas as pd
import multiprocessing as mp
import warnings
warnings.filterwarnings("ignore")

dtype_dict = {
    'hvfhs_license_num': 'object',
    'dispatching_base_num': 'object',
    'originating_base_num': 'object',
    'request_datetime': 'object',
    'on_scene_datetime': 'object',
    'pickup_datetime': 'object',
    'dropoff_datetime': 'object',
    'PULocationID': 'float64',
    'DOLocationID': 'float64',
    'trip_miles': 'float64',
    'trip_time': 'float64',
    'base_passenger_fare': 'float64',
    'tolls': 'float64',
    'bcf': 'float64',
    'sales_tax': 'float64',
    'congestion_surcharge': 'float64',
    'airport_fee': 'float64',
    'tips': 'float64',
    'driver_pay': 'float64',
    'shared_request_flag': 'object',
    'shared_match_flag': 'object',
    'access_a_ride_flag': 'object',
    'wav_request_flag': 'object',
    'wav_match_flag': 'object'
}

df = pd.read_csv("./2021 Dataset/reduced_2021.csv", dtype=dtype_dict)
df

In [ ]:
def get_start_end_dates(df):

    df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
    start_date = df['pickup_datetime'].min().strftime('%d-%m-%Y')
    end_date = df['pickup_datetime'].max().strftime('%d-%m-%Y')
    return (start_date, end_date)

start_date, end_date = get_start_end_dates(df)
print(f"Start Date: {start_date}")
print(f"End Date: {end_date}")

In [ ]:
df.tail(10)

In [ ]:
df.info()

In [ ]:
print(len(df.index))

In [ ]:
df_with_nan = df[df.isnull().any(axis=1)]
print(len(df_with_nan))

In [ ]:
df = df.drop(columns=['dispatching_base_num', 'on_scene_datetime', 'request_datetime', 'trip_time', 'originating_base_num', 'tolls', 'sales_tax', 'congestion_surcharge', 'airport_fee', 'shared_request_flag', 'shared_match_flag', 'tips', 'bcf', 'airport_fee', 'access_a_ride_flag', 'wav_request_flag', 'wav_match_flag', 'base_passenger_fare'])
df.info()

In [ ]:
df

In [ ]:
# df = df.iloc[:12626810]
# df.tail(100)

In [ ]:
df.to_csv("./2021 Dataset/cleaned_columns.csv", sep=',')

CLEANING ROWS

In [ ]:
import pandas as pd
df = pd.read_csv("./2021 Dataset/cleaned_columns.csv", delimiter=",")

In [ ]:
df = df[~((df['PULocationID'].isin([264, 265])) | (df['DOLocationID'].isin([264, 265])))]
df

In [ ]:
print(sorted(df['PULocationID'].unique()))

In [ ]:
print(sorted(df['DOLocationID'].unique()))

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
import pandas as pd

taxi_zones = pd.read_csv("./taxi_zones.csv", delimiter=",")

df = pd.merge(df, taxi_zones, how='left', left_on='PULocationID', right_on='LocationID')\
        .rename(columns={'zone': 'PUZone', 'borough': 'PUBorough', 'LocationID': 'PULocationID_merged'})

df.drop(['PULocationID_merged'], axis=1, inplace=True)

df = pd.merge(df, taxi_zones, how='left', left_on='DOLocationID', right_on='LocationID')\
        .rename(columns={'zone': 'DOZone', 'borough': 'DOBorough', 'LocationID': 'DOLocationID_merged'})

df.drop(['DOLocationID_merged'], axis=1, inplace=True)
df['PULocationID'] = pd.to_numeric(df['PULocationID'], downcast='integer')
df['DOLocationID'] = pd.to_numeric(df['DOLocationID'], downcast='integer')
df = df.drop(columns=['Unnamed: 0'])
df = df.reset_index(drop=True)
df

In [ ]:
df.to_csv("./2021 Dataset/cleaned_mapped.csv",  sep=',')

VISUALIZING NEW YORK CITY

In [ ]:
!pip install geopandas matplotlib contextily
!pip install --upgrade pip

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

gdf = gpd.read_file("./taxi_zones.zip")

fig, ax = plt.subplots(figsize=(20, 20))
gdf.plot(ax=ax, alpha=0.4, edgecolor='k')

import contextily as ctx
ctx.add_basemap(ax, crs=gdf.crs.to_string(), source=ctx.providers.CartoDB.Positron)

plt.show()

VISUALIZING THE BOROUGHS OF NEW YORK CITY

In [ ]:
print(gdf['borough'].unique())

In [ ]:
borough_counts = gdf['borough'].value_counts()
print("Zones by Borough:\n", borough_counts)

In [ ]:
import contextily as ctx

unique_boroughs = gdf['borough'].unique()

for borough in unique_boroughs:
    borough_zones = gdf[gdf['borough'] == borough]
    
    fig, ax = plt.subplots(figsize=(20, 20))
    borough_zones.plot(ax=ax, alpha=0.5, edgecolor='k')
    
    ctx.add_basemap(ax, crs=borough_zones.crs.to_string(), source=ctx.providers.CartoDB.Positron)
    
    ax.set_title(f"{borough} Taxi Zones")
    
    plt.show()

In [ ]:
print(gdf.info())
print(gdf.columns)

In [ ]:
import pandas as pd
df = pd.read_csv("./2021 Dataset/cleaned_mapped.csv", delimiter=",")
df

In [ ]:
print(df.columns)

In [ ]:
from shapely.geometry import LineString

gdf['centroid'] = gdf.centroid

location_to_centroid = gdf.set_index('LocationID')['centroid'].to_dict()

df_filtered = df[df['PULocationID'].isin(location_to_centroid.keys()) & df['DOLocationID'].isin(location_to_centroid.keys())]

df['line'] = df_filtered.apply(lambda row: LineString([location_to_centroid[row['PULocationID']], location_to_centroid[row['DOLocationID']]]), axis=1)

lines_gdf = gpd.GeoDataFrame(df, geometry='line', crs=gdf.crs)

PASSENGER FLOW FOR 2022 NYC (ALL BRANDS)

In [ ]:
fig, ax = plt.subplots(figsize=(1000, 1000))
gdf.plot(ax=ax, alpha=0.4, edgecolor='k')
lines_gdf.plot(ax=ax, linewidth=1, color='red')

ctx.add_basemap(ax, crs=gdf.crs.to_string(), source=ctx.providers.CartoDB.Positron)
plt.show()

VISUALIZING HEATMAP FOR NYC TRIPS 2022

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

trip_counts = df.groupby(['PULocationID', 'DOLocationID']).size().reset_index(name='TripCount')

pivot_table = trip_counts.pivot('PULocationID', 'DOLocationID', 'TripCount')

plt.figure(figsize=(50, 50))
sns.heatmap(pivot_table, cmap='Reds', linewidths=.5)

plt.title('Heatmap of Trips between Pickup and Drop-off Locations')
plt.xlabel('Drop-off Location ID')
plt.ylabel('Pickup Location ID')
plt.show()


MOST TRAVELLED AND LEAST TRAVELLED ROUTES IN NYC 2022

In [ ]:
df_filtered = df[df['PULocationID'] != df['DOLocationID']]

trip_counts_filtered = df_filtered.groupby(['PULocationID', 'DOLocationID']).size().reset_index(name='TripCount')

most_traveled_filtered = trip_counts_filtered.loc[trip_counts_filtered['TripCount'].idxmax()]

least_traveled_filtered = trip_counts_filtered.loc[trip_counts_filtered['TripCount'].idxmin()]

print("Most Traveled Trip Line:")
print(most_traveled_filtered)

print("Least Traveled Trip Line:")
# Doesn't really matter much
print(least_traveled_filtered)

MOST COMMON PICKUP AND DROPOFF LOCATIONS

In [ ]:
most_common_dolocation_id = df['DOLocationID'].value_counts().idxmax()
most_common_pulocation_id = df['PULocationID'].value_counts().idxmax()

most_common_dolocation_count = df['DOLocationID'].value_counts().max()
most_common_pulocation_count = df['PULocationID'].value_counts().max()

most_common_pulocation_name = df[df['PULocationID'] == most_common_pulocation_id]['PUZone'].iloc[0]
most_common_dolocation_name = df[df['DOLocationID'] == most_common_dolocation_id]['DOZone'].iloc[0]

print(f"The most common pick-up location is {most_common_pulocation_name} (ID {most_common_pulocation_id}) : {most_common_pulocation_count}.")
print(f"The most common drop-off location is {most_common_dolocation_name} (ID {most_common_dolocation_id}) : {most_common_dolocation_count}.")

SPLITTING THE DATASET INTO BRAND-WISE SUBSETS

In [ ]:
import pandas as pd
df = pd.read_parquet("./2021 Dataset/cleaned_mapped.csv", delimiter=",")
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df

In [ ]:
df['hvfhs_license_num'].unique()

In [ ]:
df_hv0003 = df[df['hvfhs_license_num'] == 'HV0003']
df_hv0003.drop(columns=['hvfhs_license_num'], inplace=True)
df_hv0005 = df[df['hvfhs_license_num'] == 'HV0005']
df_hv0005.drop(columns=['hvfhs_license_num'], inplace=True)

In [ ]:
df_hv0003

In [ ]:
df_hv0005

In [ ]:
df_hv0003.to_parquet("./2021 Dataset/Uber.parquet")
df_hv0005.to_parquet("./2021 Dataset/Lyft.parquet")
df.to_parquet("./2021 Dataset/Uber+Lyft.parquet")

In [1]:
import pandas as pd
df = pd.read_parquet("2021 Dataset\Lyft.parquet")
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11429380 entries, 0 to 42066327
Data columns (total 10 columns):
 #   Column            Dtype  
---  ------            -----  
 0   pickup_datetime   object 
 1   dropoff_datetime  object 
 2   PULocationID      int64  
 3   DOLocationID      int64  
 4   trip_miles        float64
 5   driver_pay        float64
 6   PUZone            object 
 7   PUBorough         object 
 8   DOZone            object 
 9   DOBorough         object 
dtypes: float64(2), int64(2), object(6)
memory usage: 959.2+ MB
